<a href="https://colab.research.google.com/github/pasquale90/mthesis/blob/master/Verification_method_1%5Bpercentage_aggreement%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title import gdrive
from google.colab import drive
drive.mount('/content/gdrive')
#root_path = 'gdrive/My Drive/dissertation/dcase_dataset/'

Mounted at /content/gdrive


In [0]:
import pandas as pd

#read annotations_file
annotations = 'gdrive/My Drive/dissertation/dcase_dataset/annotations.csv'
raw_annotations = pd.read_csv(annotations)

In [0]:
raw_annotations.shape

(56780, 78)

In [0]:
pd.read_csv(annotations)

,split,sensor_id,audio_filename,annotator_id,borough,block,latitude,longitude,year,week,day,hour,1-1_small-sounding-engine_presence,1-2_medium-sounding-engine_presence,1-3_large-sounding-engine_presence,1-X_engine-of-uncertain-size_presence,2-1_rock-drill_presence,2-2_jackhammer_presence,2-3_hoe-ram_presence,2-4_pile-driver_presence,2-X_other-unknown-impact-machinery_presence,3-1_non-machinery-impact_presence,4-1_chainsaw_presence,4-2_small-medium-rotating-saw_presence,4-3_large-rotating-saw_presence,4-X_other-unknown-powered-saw_presence,5-1_car-horn_presence,5-2_car-alarm_presence,5-3_siren_presence,5-4_reverse-beeper_presence,5-X_other-unknown-alert-signal_presence,6-1_stationary-music_presence,6-2_mobile-music_presence,6-3_ice-cream-truck_presence,6-X_music-from-uncertain-source_presence,7-1_person-or-small-group-talking_presence,7-2_person-or-small-group-shouting_presence,7-3_large-crowd_presence,7-4_amplified-speech_presence,7-X_other-unknown-human-voice_presence,8-1_dog-barking-whining_presence,1-1_small-sounding-engine_proximity,1-2_medium-sounding-engine_proximity,1-3_large-sounding-engine_proximity,1-X_engine-of-uncertain-size_proximity,2-1_rock-drill_proximity,2-2_jackhammer_proximity,2-3_hoe-ram_proximity,2-4_pile-driver_proximity,2-X_other-unknown-impact-machinery_proximity,3-1_non-machinery-impact_proximity,4-1_chainsaw_proximity,4-2_small-medium-rotating-saw_proximity,4-3_large-rotating-saw_proximity,4-X_other-unknown-powered-saw_proximity,5-1_car-horn_proximity,5-2_car-alarm_proximity,5-3_siren_proximity,5-4_reverse-beeper_proximity,5-X_other-unknown-alert-signal_proximity,6-1_stationary-music_proximity,6-2_mobile-music_proximity,6-3_ice-cream-truck_proximity,6-X_music-from-uncertain-source_proximity,7-1_person-or-small-group-talking_proximity,7-2_person-or-small-group-shouting_proximity,7-3_large-crowd_proximity,7-4_amplified-speech_proximity,7-X_other-unknown-human-voice_proximity,8-1_dog-barking-whining_proximity,1_engine_presence,2_machinery-impact_presence,3_non-machinery-impact_presence,4_powered-saw_presence,5_alert-signal_presence,6_music_presence,7_human-voice_presence,8_dog_presence
0,validate,0,00_000066.wav,95,1,547,40.729508,-73.993881,2017,28,2,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,far,near,near,near,near,far,far,notsure,near,far,near,far,near,far,near,near,far,near,far,near,near,far,far,far,near,near,near,far,far,1,1,1,1,1,1,1,1
1,validate,0,00_000066.wav,108,1,547,40.729508,-73.993881,2017,28,2,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,near,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,0,0,0
2,validate,0,00_000066.wav,127,1,547,40.729508,-73.993881,2017,28,2,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,far,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,0,0,0
3,validate,0,00_000071.wav,64,1,547,40.729508,-73.993881,2016,50,5,13,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,notsure,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,0,0,0,0,0,0
4,validate,0,00_000071.wav,66,1,547,40.729508,-73.993881,2016,50,5,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,notsure,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56775,validate,10,10_003220.wav,-

In [0]:
print(raw_annotations.dtypes)

In [0]:
#direct separation into different dataframes
train_split = raw_annotations[raw_annotations.split == 'train']
validation_split = raw_annotations[raw_annotations.split != 'train']

ground_truth = raw_annotations[raw_annotations.annotator_id == 0]

print ('train_split_shape:', train_split.shape)
print ('validation_split_shape:', validation_split.shape)
print ('ground_truth_shape:',ground_truth.shape)

#check splits
test_t= train_split['audio_filename'].unique()
test_v= validation_split['audio_filename'].unique()
print('testing the train size',test_t.shape)#DONE SIZE=13538
print('testing the validation size',test_v.shape)#DONE SIZE=4308

train_split_shape: (41444, 78)
validation_split_shape: (15336, 78)
ground_truth_shape: (716, 78)
testing the train size (13538,)
testing the validation size (4308,)


In [0]:
#Sort by indexes
import numpy as np
train_split.reset_index(inplace=True, drop=True )
validation_split.reset_index(inplace=True, drop=True )
ground_truth.reset_index(inplace=True, drop=True )

print(validation_split.index)
print(train_split.index)

RangeIndex(start=0, stop=15336, step=1)
RangeIndex(start=0, stop=41444, step=1)


In [0]:
print (train_split.columns)

In [0]:
#@title Export CSVs
#export to csv train and validation sets

train_split.to_csv('train.csv',header=True,index=True)
validation_split.to_csv('validation.csv',header=True,index=True)
ground_truth.to_csv('ground_truth.csv',header=True,index=True)
"""
import zipfile

with zipfile.ZipFile('dcase_splits.zip', 'w') as csv_zip:
    csv_zip.writestr("train.csv",)
    csv_zip.writestr("validation.csv")
    csv_zip.writestr("ground_truth.csv")
"""

METHOD 1

In [0]:
#METHOD 1 - variables

#fine-grained labels column limits
colstart_fine=train_split.columns.get_loc('1-1_small-sounding-engine_presence')
colend_fine=train_split.columns.get_loc('8-1_dog-barking-whining_presence')

#proximity limits
colstart_prox=train_split.columns.get_loc('1-1_small-sounding-engine_proximity')
colend_prox=train_split.columns.get_loc('8-1_dog-barking-whining_proximity')

#coarse-grained labels column limits
colstart_coarse=train_split.columns.get_loc('1_engine_presence')
colend_coarse=train_split.columns.get_loc('8_dog_presence')

print(colstart_fine,colend_fine,colstart_coarse,colend_coarse,colstart_prox,colend_prox)

12 40 70 77 41 69


In [0]:
#METHOD 1 - Create init-Tables{Training and Validation}

#get labels
label_names=[]
for col in train_split.columns:
    label_names.append(col)
#print ('label_names: ', label_names)

#train_unique
train_unq=pd.DataFrame(columns=label_names)#empty table with named columns
#print ('\nTrain_unq shape:\n-still empty isnt it??',train_unq.shape)

#valid_unique
valid_unq=pd.DataFrame(columns=label_names)
#print ('\nValid_unq shape:\n-still empty isnt it??',valid_unq.shape)

#step 1 - put unique audio filenames
train_unq['audio_filename'] = train_split['audio_filename'].unique() 
valid_unq['audio_filename'] = validation_split['audio_filename'].unique()


#fill the 'split'_column values
for i in range (len(train_unq)):
  train_unq['split'].iloc[i]='train'
  if(i<len(valid_unq)):
    valid_unq['split'].iloc[i]='validate'
print('\n\nFirst fields are filled with values_train:\nIndex 1....\n',train_unq.iloc[0])
print('\n\nFirst fields are filled with values_valid:\nIndex 1....\n',valid_unq.iloc[-1])

In [0]:
fine_length=colend_fine-colstart_fine+1
print(fine_length)

prox_length=colend_prox-colstart_prox+1
print(prox_length)

coarse_length=colend_coarse-colstart_coarse+1
print(coarse_length)

label_length=fine_length+prox_length+coarse_length+1#+1 for the audiofile name as an index for every row
print(label_length)

NameError: ignored

In [0]:
#Initialize the 'fine' & 'coarse' labels with empty values{Training and Validation}
#for audiofile in range (5):
for audiofile in range (len(train_unq)):
  for label in range(colstart_fine,colend_fine+1):#Fine_labels
    #print(audiofile,label,'\n')
    train_unq.iat[audiofile,label]= 0
    if(audiofile<len(valid_unq)):
      valid_unq.iat[audiofile,label]= 0

  for label in range(colstart_coarse,colend_coarse+1):#Coarse_labels
    #print(audiofile,label,'\n')
    train_unq.iat[audiofile,label]= 0
    if(audiofile<len(valid_unq)):
      valid_unq.iat[audiofile,label]= 0

#proximity??? Should we keep files that have a near indicator only as more reliable?
#Οr do we ought to discard the files with uncertain proximity
#Where proximity labels can prove useful?

#train_unq.to_csv('check_train_unq_label values.csv',header=True,index=True) DONE
#valid_unq.to_csv('check_valid_unq_label values.csv',header=True,index=True) DONE

In [0]:
#METHOD 1 - variables_fill

#keep a counter for every different label of a file + 1 column for the filename for initialize puproses
#29 fine-grained, 29 proximity, 8 coarse-grained
train_map_counter = np.zeros(shape=(len(train_unq.index),label_length),dtype=int)
valid_map_counter = np.zeros(shape=(len(valid_unq.index),label_length),dtype=int)

print(train_map_counter.shape,type(train_map_counter))
print(valid_map_counter.shape,type(valid_map_counter))

train_backup=train_split
valid_backup=validation_split

#collect_errors = np.empty(shape=len(train_split),dtype=int)#float->real=row,decimal=col of valid table
batch_size=7000

(13538, 67) <class 'numpy.ndarray'>
(4308, 67) <class 'numpy.ndarray'>


batch 1=70000
batch 2=14000
batch 3=21000
batch 4=28000
batch 5=35000
batch 6=42000-41444

In [0]:
print(train_map_counter[0,0])

0


In [0]:
a=10
for i in range (a*2,a*4):
  print(i)

In [0]:
#METHOD 1 - filling the empty spaces of the first 12 columns{TRAIN_DATA}

#ATTEMPT TO JOIN!

#train_data
for r in range(batch_size*4,len(train_split)):#for every row in the 1st batch of the train_data
#for r in range(50):
  print('file %d out of %d.' %(r,len(train_split)))
  #print('r:%d\t'%r)
  for audiofn in range(len(train_unq.index)): #for every row in unique_recording 
    #print('audiofn:%d'%audiofn)
    if(train_split['audio_filename'].iloc[r]==train_unq['audio_filename'].iloc[audiofn]):
      train_map_counter[audiofn,0]+=1

      if(train_map_counter[audiofn,0]==1):#initialize condition
        #print('I found the unique name and it is the first time...Initiallizing...')

        train_unq['sensor_id'].iloc[audiofn]=train_split['sensor_id'].iloc[r]#sensor_id
        train_unq['borough'].iloc[audiofn]=train_split['borough'].iloc[r]#borough
        train_unq['block'].iloc[audiofn]=train_split['block'].iloc[r]#block
        train_unq['latitude'].iloc[audiofn]=train_split['latitude'].iloc[r]#latitude
        train_unq['longitude'].iloc[audiofn]=train_split['longitude'].iloc[r]#longitude
        train_unq['year'].iloc[audiofn]=train_split['year'].iloc[r]#year
        train_unq['week'].iloc[audiofn]=train_split['week'].iloc[r]#week
        train_unq['day'].iloc[audiofn]=train_split['day'].iloc[r]#day
        train_unq['hour'].iloc[audiofn]=train_split['hour'].iloc[r]#hour

      #elif(train_map_counter[audiofn,0]>1):#should be removed-->there is no use at all
        #print('I found unq but it is not the first time...Verifying.....')

      train_unq['annotator_id'].iloc[audiofn]=train_map_counter[audiofn,0]#annotation_count

#FINE-GRAINED -adding values 

      for fc in range(colstart_fine,colend_fine+1):
        #print('fc:%d\n'%fc)
        if (train_split.iat[r,fc]==1 or train_split.iat[r,fc]==1.0 or train_split.iat[r,fc]==0 or train_split.iat[r,fc]==0.0):
          #print('In row %d, i found a presensce in fc column %d, with a value of %f'%(r,fc,train_split.iat[r,fc]))#an to ena i to allo -->count++
          train_map_counter[audiofn,fc-11]+=1
          #print('train_map_counter[audiofn,fc-11]+=1\t%d'%train_map_counter[audiofn,fc-11])
          train_unq.iat[audiofn,fc]+=train_split.iat[r,fc].astype(int)

#COARSE-GRAINED -adding values 

      for cc in range(colstart_coarse,colend_coarse+1):
        #print(cc)
        if (train_split.iat[r,cc]==1 or train_split.iat[r,cc]==1.0 or train_split.iat[r,cc]==0 or train_split.iat[r,cc]==0.0):
          #print ('In row %d, i found a presensce in cc column %d'%(r,cc))#an to ena i to allo -->count++
          train_map_counter[audiofn,cc-11]+=1
          train_unq.iat[audiofn,cc]+=train_split.iat[r,cc].astype(int)







train_unq.to_csv('check_unq.csv',header=True,index=True)
train_split.to_csv('check_train_split.csv',header=True,index=True)
#np.savetxt('check_mapcounter.csv', train_map_counter, delimiter=",")
trmc=pd.DataFrame(train_map_counter)
trmc.to_csv('check_mapcounter.csv',header=True,index=True)

In [0]:
#METHOD 1 - filling the empty spaces of the first 12 columns{VALIDATION_DATA}

#ATTEMPT TO JOIN!

#Validation Data
for r in range(len(validation_split)):#for every row in the 1st batch of the train_data
#for r in range(50):
  print('file %d out of %d.' %(r,len(validation_split)))
  #print('r:%d\t'%r)
  for audiofn in range(len(valid_unq.index)): #for every row in unique_recording 
    #print('audiofn:%d'%audiofn)
    if(validation_split['audio_filename'].iloc[r]==valid_unq['audio_filename'].iloc[audiofn]):
      valid_map_counter[audiofn,0]+=1

      if(valid_map_counter[audiofn,0]==1):#initialize condition
        #print('I found the unique name and it is the first time...Initiallizing...')

        valid_unq['sensor_id'].iloc[audiofn]=validation_split['sensor_id'].iloc[r]#sensor_id
        valid_unq['borough'].iloc[audiofn]=validation_split['borough'].iloc[r]#borough
        valid_unq['block'].iloc[audiofn]=validation_split['block'].iloc[r]#block
        valid_unq['latitude'].iloc[audiofn]=validation_split['latitude'].iloc[r]#latitude
        valid_unq['longitude'].iloc[audiofn]=validation_split['longitude'].iloc[r]#longitude
        valid_unq['year'].iloc[audiofn]=validation_split['year'].iloc[r]#year
        valid_unq['week'].iloc[audiofn]=validation_split['week'].iloc[r]#week
        valid_unq['day'].iloc[audiofn]=validation_split['day'].iloc[r]#day
        valid_unq['hour'].iloc[audiofn]=validation_split['hour'].iloc[r]#hour

      #elif(valid_map_counter[audiofn,0]>1):#should be removed-->there is no use at all
        #print('I found unq but it is not the first time...Verifying.....')
        
      valid_unq['annotator_id'].iloc[audiofn]=valid_map_counter[audiofn,0]#annotation_count

#FINE-GRAINED -adding values 

      for fc in range(colstart_fine,colend_fine+1):
        #print('fc:%d\n'%fc)
        if (validation_split.iat[r,fc]==1 or validation_split.iat[r,fc]==1.0 or validation_split.iat[r,fc]==0 or validation_split.iat[r,fc]==0.0):
          #print('In row %d, i found a presensce in fc column %d, with a value of %f'%(r,fc,validation_split.iat[r,fc]))#an to ena i to allo -->count++
          valid_map_counter[audiofn,fc-11]+=1
          #print('valid_map_counter[audiofn,fc-11]+=1\t%d'%valid_map_counter[audiofn,fc-11])
          valid_unq.iat[audiofn,fc]+=validation_split.iat[r,fc].astype(int)

#COARSE-GRAINED -adding values 

      for cc in range(colstart_coarse,colend_coarse+1):
        #print(cc)
        if (validation_split.iat[r,cc]==1 or validation_split.iat[r,cc]==1.0 or validation_split.iat[r,cc]==0 or validation_split.iat[r,cc]==0.0):
          #print ('In row %d, i found a presensce in cc column %d'%(r,cc))#an to ena i to allo -->count++
          valid_map_counter[audiofn,cc-11]+=1
          valid_unq.iat[audiofn,cc]+=validation_split.iat[r,cc].astype(int)

valid_unq.to_csv('check_valunq.csv',header=True,index=True)
validation_split.to_csv('check_valalidation.csv',header=True,index=True)
#np.savetxt('check_mapcounter.csv', train_map_counter, delimiter=",")
vmc=pd.DataFrame(valid_map_counter)
vmc.to_csv('check_valid_mapcounter.csv',header=True,index=True)

In [0]:
#@title filling the empty spaces of the first 12 columns{TRAIN_DATA}
#METHOD 1 - filling the empty spaces of the first 12 columns{TRAIN_DATA}

#train_data
#for r in range(batch_size):#for every row in the 1st batch of the train_data
for r in range(50):
  #print('file %d out of %d.' %(r,len(train_split)))
  print('r:%d\t'%r)
  for audiofn in range(len(train_unq.index)): #for every row in unique_recording 
    #print('audiofn:%d'%audiofn)
    if(train_split['audio_filename'].iloc[r]==train_unq['audio_filename'].iloc[audiofn]):
      train_map_counter[audiofn,0]+=1

      if(train_map_counter[audiofn,0]==1):#initialize condition
        print('I found the unique name and it is the first time...Initiallizing...')

        train_unq['sensor_id'].iloc[audiofn]=train_split['sensor_id'].iloc[r]#sensor_id
        train_unq['borough'].iloc[audiofn]=train_split['borough'].iloc[r]#borough
        train_unq['block'].iloc[audiofn]=train_split['block'].iloc[r]#block
        train_unq['latitude'].iloc[audiofn]=train_split['latitude'].iloc[r]#latitude
        train_unq['longitude'].iloc[audiofn]=train_split['longitude'].iloc[r]#longitude
        train_unq['year'].iloc[audiofn]=train_split['year'].iloc[r]#year
        train_unq['week'].iloc[audiofn]=train_split['week'].iloc[r]#week
        train_unq['day'].iloc[audiofn]=train_split['day'].iloc[r]#day
        train_unq['hour'].iloc[audiofn]=train_split['hour'].iloc[r]#hour

      elif(train_map_counter[audiofn,0]>1):#should be removed-->there is no use at all
        print('I found unq but it is not the first time...Verifying.....')

      train_unq['annotator_id'].iloc[audiofn]=train_map_counter[audiofn,0]#annotation_count
    
train_unq.to_csv('check_unq_12col.csv',header=True,index=True)
train_split.to_csv('check_tr_Split_12col.csv',header=True,index=True)
#np.savetxt('check_mapcounter.csv', train_map_counter, delimiter=",")
#trmc=pd.DataFrame(train_map_counter)
#trmc.to_csv('check_mapcounter.csv',header=True,index=True)

'''
print("\ntrain_map-1st row",train_map_counter[0])
print("\ntrain_map-1st row",train_map_counter[1])
print("\ntrain_map-1st row",train_map_counter[2])
print(train_unq.loc[0:2,['audio_filename','sensor_id', 'borough', 'block', 'latitude', 'longitude', 'year', 'week', 'day', 'hour']])
'''

In [0]:
#@title filling the empty spaces of the first 12 columns{VALIDATION_DATA}
#METHOD 1 - filling the empty spaces of the first 12 columns{VALIDATION_DATA}

#Validation Data
#for r in range(batch_size):#for every row in the 1st batch of the train_data
for r in range(50):
  #print('file %d out of %d.' %(r,len(train_split)))
  print('r:%d\t'%r)
  for audiofn in range(len(valid_unq.index)): #for every row in unique_recording 
    #print('audiofn:%d'%audiofn)
    if(validation_split['audio_filename'].iloc[r]==valid_unq['audio_filename'].iloc[audiofn]):
      valid_map_counter[audiofn,0]+=1

      if(valid_map_counter[audiofn,0]==1):#initialize condition
        print('I found the unique name and it is the first time...Initiallizing...')

        valid_unq['sensor_id'].iloc[audiofn]=validation_split['sensor_id'].iloc[r]#sensor_id
        valid_unq['borough'].iloc[audiofn]=validation_split['borough'].iloc[r]#borough
        valid_unq['block'].iloc[audiofn]=validation_split['block'].iloc[r]#block
        valid_unq['latitude'].iloc[audiofn]=validation_split['latitude'].iloc[r]#latitude
        valid_unq['longitude'].iloc[audiofn]=validation_split['longitude'].iloc[r]#longitude
        valid_unq['year'].iloc[audiofn]=validation_split['year'].iloc[r]#year
        valid_unq['week'].iloc[audiofn]=validation_split['week'].iloc[r]#week
        valid_unq['day'].iloc[audiofn]=validation_split['day'].iloc[r]#day
        valid_unq['hour'].iloc[audiofn]=validation_split['hour'].iloc[r]#hour

      elif(valid_map_counter[audiofn,0]>1):#should be removed-->there is no use at all
        print('I found unq but it is not the first time...Verifying.....')
        
      valid_unq['annotator_id'].iloc[audiofn]=valid_map_counter[audiofn,0]#annotation_count
    
valid_unq.to_csv('check_val_12col.csv',header=True,index=True)
validation_split.to_csv('check_val_Split_12col.csv',header=True,index=True)
#np.savetxt('check_mapcounter.csv', train_map_counter, delimiter=",")
#trmc=pd.DataFrame(train_map_counter)
#trmc.to_csv('check_mapcounter.csv',header=True,index=True)

'''
print("\ntrain_map-1st row",train_map_counter[0])
print("\ntrain_map-1st row",train_map_counter[1])
print("\ntrain_map-1st row",train_map_counter[2])
print(train_unq.loc[0:2,['audio_filename','sensor_id', 'borough', 'block', 'latitude', 'longitude', 'year', 'week', 'day', 'hour']])
'''

In [0]:
#Now divide and conquer! 
#Get the results
#extract mfccs/mel spectograms
#Make a simple network 
#compare results

In [0]:
#discard ground truth from train and validation splits

#graphs

#percentage

#preperation for method 2

In [0]:
#@title import gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Division OF Annotation_Sum with map_counter {TRAINING DATA}
import pandas as pd

#read annotations_file
tunq = 'gdrive/My Drive/dissertation/dcase_dataset/method1/train/train_unq.csv'
train_unq = pd.read_csv(tunq,sep="\;")
train_unq=train_unq.drop(train_unq.columns[0],axis=1)

print (train_unq.shape)
#print(train_unq.columns)

#read train_map_counter
tmc = 'gdrive/My Drive/dissertation/dcase_dataset/method1/train/train_mapcounter.csv'
train_map_counter = pd.read_csv(tmc,sep="\;",comment='\"')
train_map_counter=train_map_counter.drop(train_map_counter.columns[0],axis=1)
#train_map_counter=train_map_counter.drop(train_map_counter.columns[67],axis=1)

print (train_map_counter.shape)
#print(train_map_counter.columns)

#Divide 
train_verified_percentage=train_unq

for r in range(len(train_unq)):
  for fc in range (colstart_fine,colend_fine+1):#fine
    train_verified_percentage.iat[r,fc]=(train_unq.iat[r,fc]/train_map_counter.iat[r,fc-11])*100
    
  for cc in range (colstart_coarse,colend_coarse+1):#coarse
    train_verified_percentage.iat[r,cc]=(train_unq.iat[r,cc]/train_map_counter.iat[r,cc-11])*100  

#Export CSV
#train_verified_percentage.to_csv("train_verified_percentage.csv",header=True,index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


(13538, 78)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


(13538, 67)


In [0]:
#Verified_Labels_1
#The Labels that hold a value greater than the specified thresshold


In [0]:
#Division OF Annotation Sum with map_counter {VALIDATION DATA}

#read annotations_file
vunq = 'gdrive/My Drive/dissertation/dcase_dataset/method1/validation/valid_unq.csv'
valid_unq = pd.read_csv(vunq,sep="\,")
valid_unq=valid_unq.drop(valid_unq.columns[0],axis=1)

print (valid_unq.shape)
#print(valid_unq.columns)

#read valid_map_counter
vmc = 'gdrive/My Drive/dissertation/dcase_dataset/method1/validation/valid_mapcounter.csv'
valid_map_counter = pd.read_csv(vmc,sep="\,")
valid_map_counter=valid_map_counter.drop(valid_map_counter.columns[0],axis=1)
#valid_map_counter=valid_map_counter.drop(valid_map_counter.columns[67],axis=1)

print (valid_map_counter.shape)
#print(valid_map_counter.columns)

#Divide 
valid_verified_percentage=valid_unq

for r in range(len(valid_unq)):
  for fc in range (colstart_fine,colend_fine+1):#fine
    valid_verified_percentage.iat[r,fc]=(valid_unq.iat[r,fc]/valid_map_counter.iat[r,fc-11])*100

  for cc in range (colstart_coarse,colend_coarse+1):#coarse
    valid_verified_percentage.iat[r,cc]=(valid_unq.iat[r,cc]/valid_map_counter.iat[r,cc-11])*100  

#Export CSV
#valid_verified_percentage.to_csv("valid_verified_percentage.csv",header=True,index=False)



In [0]:
#Get if 50% of annotations: possitive, else:absense
#train_verified_percentage,valid_verified_percentage
train_verified=train_verified_percentage
valid_verified=valid_verified_percentage

for r in range (len(train_unq)):
  for fc in range (colstart_fine,colend_fine+1):#fine
    if (train_verified_percentage.iat[r,fc]>49):
      train_verified.iat[r,fc]=1
    else:
      train_verified.iat[r,fc]=0
    


  for cc in range (colstart_coarse,colend_coarse+1):#coarse
    if (train_verified_percentage.iat[r,cc]>49):
      train_verified.iat[r,cc]=1
    else:
      train_verified.iat[r,cc]=0

  if(r<len(valid_unq)):
    for fc in range (colstart_fine,colend_fine+1):#fine
      if (valid_verified_percentage.iat[r,fc]>49):
        valid_verified.iat[r,fc]=1
      else:
        valid_verified.iat[r,fc]=0

    for cc in range (colstart_coarse,colend_coarse+1):#coarse
      if (valid_verified_percentage.iat[r,cc]>49):
        valid_verified.iat[r,cc]=1
      else:
        valid_verified.iat[r,cc]=0

#train_verified.to_csv("train_verified_method1.csv",header=True,index=False)
#valid_verified.to_csv("valid_verified_method1.csv",header=True,index=False)

In [0]:
#Drop uncertain values {train_data}

print(train_verified.shape)
print(valid_verified.shape)

train_verified_counter=pd.Series(np.zeros(len(train_verified)))
#print(train_verified_counter.shape)
#print(train_verified_counter)

train_verified_copy=train_verified.copy()

for r in range (len(train_verified)):
  for fc in range (colstart_fine,colend_fine+1):#fine
    if (train_verified.iat[r,fc]==1):
      train_verified_counter[r]+=1
  for cc in range (colstart_coarse,colend_coarse+1):#coarse
    if (train_verified.iat[r,cc]==1):
      train_verified_counter[r]+=1
  #print("%s, counter of verified labels = %d"%(train_verified['audio_filename'].iloc[r],train_verified_counter[r]))
  if (train_verified_counter[r]==0):
    print("dropping %s file, as  it' s counter of verified labels = %d"%(train_verified['audio_filename'].iloc[r],train_verified_counter[r]))
    train_verified_copy=train_verified_copy.drop(train_verified.index[r],axis=0)

print(train_verified.shape)
print(train_verified_copy.shape)

print("Droped %d files out of train data according to verification technique 1 "%(len(train_unq)-len(train_verified_copy)))  

(13538, 78)
(4308, 78)
dropping 02_010255.wav file, as  it' s counter of verified labels = 0
dropping 11_010297.wav file, as  it' s counter of verified labels = 0
dropping 11_010395.wav file, as  it' s counter of verified labels = 0
dropping 11_010657.wav file, as  it' s counter of verified labels = 0
dropping 18_010519.wav file, as  it' s counter of verified labels = 0
dropping 19_010483.wav file, as  it' s counter of verified labels = 0
dropping 20_010811.wav file, as  it' s counter of verified labels = 0
dropping 24_010739.wav file, as  it' s counter of verified labels = 0
dropping 24_010788.wav file, as  it' s counter of verified labels = 0
dropping 27_010277.wav file, as  it' s counter of verified labels = 0
dropping 29_010442.wav file, as  it' s counter of verified labels = 0
dropping 32_010383.wav file, as  it' s counter of verified labels = 0
dropping 35_010586.wav file, as  it' s counter of verified labels = 0
dropping 01_000497.wav file, as  it' s counter of verified labels =

In [0]:
train_verified_copy.to_csv("/content/gdrive/My Drive/dissertation/dcase_dataset/method1/train_labels[verified_1].csv",header=True,index=False)
valid_verified_copy.to_csv("/content/gdrive/My Drive/dissertation/dcase_dataset/method1/valid_labels[verified_1].csv",header=True,index=False)

In [0]:
#Drop uncertain values {validation data}
print(valid_verified.shape)

valid_verified_counter=pd.Series(np.zeros(len(train_verified)))
#print(train_verified_counter.shape)
#print(train_verified_counter)

valid_verified_copy=valid_verified.copy()

for r in range (len(valid_verified)):
  for fc in range (colstart_fine,colend_fine+1):#fine
    if (valid_verified.iat[r,fc]==1):
      valid_verified_counter[r]+=1
  for cc in range (colstart_coarse,colend_coarse+1):#coarse
    if (valid_verified.iat[r,cc]==1):
      valid_verified_counter[r]+=1
  if (valid_verified_counter[r]==0):
    print("dropping %s file, as  it' s counter of verified labels = %d"%(valid_verified['audio_filename'].iloc[r],valid_verified_counter[r]))
    valid_verified_copy=valid_verified_copy.drop(valid_verified.index[r],axis=0)

print(valid_verified.shape)
print(valid_verified_copy.shape)

print("Droped %d files out of validation data according to verification technique 1 "%(len(valid_unq)-len(valid_verified_copy)))
  

(4308, 78)
dropping 00_000071.wav file, as  it' s counter of verified labels = 0
dropping 00_000085.wav file, as  it' s counter of verified labels = 0
dropping 00_000179.wav file, as  it' s counter of verified labels = 0
dropping 00_000901.wav file, as  it' s counter of verified labels = 0
dropping 00_000925.wav file, as  it' s counter of verified labels = 0
dropping 00_001392.wav file, as  it' s counter of verified labels = 0
dropping 00_001448.wav file, as  it' s counter of verified labels = 0
dropping 00_001460.wav file, as  it' s counter of verified labels = 0
dropping 00_002112.wav file, as  it' s counter of verified labels = 0
dropping 00_002185.wav file, as  it' s counter of verified labels = 0
dropping 00_002305.wav file, as  it' s counter of verified labels = 0
dropping 00_002383.wav file, as  it' s counter of verified labels = 0
dropping 00_002445.wav file, as  it' s counter of verified labels = 0
dropping 00_002451.wav file, as  it' s counter of verified labels = 0
dropping 